In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

pre_trained_model = tf.keras.applications.ResNet50(
    include_top=False, weights="imagenet",
    input_shape=(256, 256, 3), classes=6
)

for layer in pre_trained_model.layers:
  layer.trainable = True
  
# pre_trained_model.summary()



94773248/94765736 [==============================] - 118s 1us/step


In [2]:
training_dir = './images2/training/'

In [3]:
x = pre_trained_model.output
# creating model
x = tf.keras.layers.Flatten()(x)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = tf.keras.layers.Dense  (6, activation='softmax')(x)           
model = tf.keras.models.Model( pre_trained_model.input, x) 

In [13]:
model.summary()

_______________
conv4_block2_preact_bn (BatchNo (None, 16, 16, 1024) 4096        conv4_block1_out[0][0]           
__________________________________________________________________________________________________
conv4_block2_preact_relu (Activ (None, 16, 16, 1024) 0           conv4_block2_preact_bn[0][0]     
__________________________________________________________________________________________________
conv4_block2_1_conv (Conv2D)    (None, 16, 16, 256)  262144      conv4_block2_preact_relu[0][0]   
__________________________________________________________________________________________________
conv4_block2_1_bn (BatchNormali (None, 16, 16, 256)  1024        conv4_block2_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block2_1_relu (Activation (None, 16, 16, 256)  0           conv4_block2_1_bn[0][0]          
_____________________________________________________________________________________________

In [4]:
class StopCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc') > .98 and logs.get('val_acc') > .98):
      print("objective reached, stoping training ...")
      self.model.stop_training = True

stop_callback = StopCallback()

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(learning_rate=0.0001),
              metrics=['acc'])

In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 180,
    width_shift_range = .2,
    height_shift_range = .2,
    shear_range = .6,
    zoom_range = .6,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
    validation_split=.2,
)
train_generator = datagen.flow_from_directory(
    training_dir,
    target_size = (256, 256),
    batch_size = 10,
    class_mode = 'categorical',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    training_dir,
    target_size = (256, 256),
    batch_size = 5,
    class_mode = 'categorical',
    subset='validation'
)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Found 557 images belonging to 6 classes.
Found 137 images belonging to 6 classes.


In [20]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size,  
      epochs=30,
      verbose=2,
      validation_data = validation_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      callbacks=[stop_callback])

Epoch 1/20
111/111 - 230s - loss: 0.2589 - acc: 0.9040 - val_loss: 0.8810 - val_acc: 0.8741
Epoch 2/20
111/111 - 233s - loss: 0.3058 - acc: 0.8913 - val_loss: 0.2816 - val_acc: 0.9185
Epoch 3/20
111/111 - 232s - loss: 0.2404 - acc: 0.9112 - val_loss: 0.3936 - val_acc: 0.8593
Epoch 4/20
111/111 - 233s - loss: 0.2736 - acc: 0.9203 - val_loss: 0.7343 - val_acc: 0.8963
Epoch 5/20
111/111 - 232s - loss: 0.2717 - acc: 0.9203 - val_loss: 0.3821 - val_acc: 0.9037
Epoch 6/20
111/111 - 222s - loss: 0.2389 - acc: 0.9257 - val_loss: 0.4716 - val_acc: 0.8741
Epoch 7/20
111/111 - 211s - loss: 0.2527 - acc: 0.9130 - val_loss: 0.4555 - val_acc: 0.8963
Epoch 8/20
111/111 - 211s - loss: 0.2259 - acc: 0.9167 - val_loss: 0.2790 - val_acc: 0.8741
Epoch 9/20
111/111 - 211s - loss: 0.2837 - acc: 0.9149 - val_loss: 1.4954 - val_acc: 0.7852
Epoch 10/20
111/111 - 198s - loss: 0.3087 - acc: 0.9004 - val_loss: 0.5519 - val_acc: 0.8741
Epoch 11/20
111/111 - 198s - loss: 0.2188 - acc: 0.9297 - val_loss: 0.8268 - va

KeyboardInterrupt: 

In [ ]:
model.save('./model3.h5')